In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
from tqdm.auto import tqdm

#gen a file path list with all .fits files in a directory
master_table = Table.read("master_table.fits")
print("Total objects in master table:", len(master_table))
for obj in master_table:
    r = obj['distance']
    ha=obj['ha'];ha_limit=obj['ha_limit']
    hb=obj['hb'];hb_limit=obj['hb_limit']
    mask_r = (r < 1*obj['re'])
    mask_sb = (ha > ha_limit) & (hb > hb_limit)
    if len(r[mask_r & mask_sb]) >= 2:
            obj['has_profile'] = 1
    else:
            obj['has_profile'] = 0
print("initial Objects with profile:", len(master_table[master_table['has_profile'] == 1]))

from pathlib import Path
import re
#open folder, match  {subfield}_{id}.png return subfield, id pair
def extract_subfield_id(folder_name):
    folder_path = Path(folder_name)
    png_files = sorted(folder_path.glob('*.png'))
    subfield_id_pairs = []
    pattern = r'([a-z0-9]+)_(\d+)\.png'
    for filename in png_files:
        match = re.match(pattern, filename.name)
        if match:
            subfield = match.group(1)
            id_num = match.group(2)
            subfield_id_pairs.append((subfield, id_num))
    return subfield_id_pairs


folder = 'output/plots/manual_selected'
subfield_id_list = extract_subfield_id(folder)
manual_select = []
i = 0
for obj in master_table:
    if (obj['subfield'].lower(), str(obj['id']).zfill(5)) in subfield_id_list:
        manual_select.append(-99) # Placeholder for manual selection
        i += 1
    else:
        manual_select.append(1)  # Not selected
if i == len(subfield_id_list):
    print("All manual_selected objects matched.")
master_table['manual_select'] = manual_select

folder = 'output/plots/need_seg'
subfield_id_list = extract_subfield_id(folder)
i = 0
for j, obj in enumerate(master_table):
    if (obj['subfield'].lower(), str(obj['id']).zfill(5)) in subfield_id_list:
        master_table['manual_select'][j] = 0  # Mark as need segmentation
        i += 1
if i == len(subfield_id_list):
    print("All need_seg objects matched.")

selected = master_table[(master_table['manual_select'] == 1) & (master_table['has_profile'] ==1)]
print("Selected objects(before seg):", len(selected))
selected.write("selected_objects.fits", overwrite=True)

Total objects in master table: 1418
initial Objects with profile: 332
All manual_selected objects matched.
All need_seg objects matched.
Selected objects(before seg): 239


All need_seg objects matched.


# Test of diagnostic plots

In [ ]:
from astropy.visualization import ImageNormalize, PercentileInterval, AsinhStretch
import astropy.units as u
from matplotlib import cm
import numpy as np
from photutils.aperture import EllipticalAperture
from astropy.visualization import LogStretch, ImageNormalize
from utils.colors import color1, color2, color3
%matplotlib inline
from matplotlib import pyplot as plt

def plot_for_obj(extracted_fits_path, profile_fits_path):
    with fits.open(profile_fits_path) as hdul:
        obj = Table(hdul[1].data)[0]
        r=obj['distance']/0.1 * obj['pixel_length']
        ha=obj['ha']; ha_err=obj['ha_err']; ha_lim=obj['ha_limit']
        hb=obj['hb']; hb_err=obj['hb_err']; hb_lim=obj['hb_limit']
        balmer = obj['balmer']; balmer_err = obj['balmer_err']
        
    with fits.open(extracted_fits_path) as hdul:
        
        re = obj['re']/0.1; q = obj['q']; pa = obj['pa']
        
        #prepare segmentation map
        seg = hdul[2].data == obj['ID']
        #prepare 2D Balmer map
        
        plt.figure(figsize=(15,10))        
        for plot_index, hdul_index, name in [
                (1, 3, f"{hdul[3].header['FILTER']}"),
                (2, 4, 'Hα'),
                (3, 6, 'Hβ')]:
            ax = plt.subplot(2, 3, plot_index)
            ax.tick_params(axis='both', labelbottom=False, direction='in', which='both', 
            top=True, right=True, left=True, bottom=True, labelsize=13)
            
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            
            ax.text(
                0.02, 0.98, f"{name}", transform=ax.transAxes,
                fontsize=18, color='white', va='top', ha='left',
                bbox=dict(facecolor='black', alpha=0.3, edgecolor='none', pad=2)
            )
            data = hdul[hdul_index].data
            # avoid zeros and negatives for log stretch
            norm = ImageNormalize(np.where(data > 0, data, np.nan), interval=PercentileInterval(99.5), stretch=LogStretch())

            # plot the segmentation map
            viridis_cmap = plt.get_cmap('viridis')
            grey_viridis_cmap = cm.colors.ListedColormap(
                np.mean(viridis_cmap(np.linspace(0, 1, 256))[:, :3], axis=1, keepdims=True).repeat(3, axis=1))
            grey_viridis_cmap._init()

            ax.imshow(np.where(seg, data, np.nan), origin='lower', norm=norm, cmap='viridis')
            ax.imshow(np.where(~seg, data, np.nan), origin='lower', norm=norm, cmap=grey_viridis_cmap, alpha=0.7)
            # set the title


            # plot the ellipse at the effective radius for all images, use photutils aperture
            center = (data.shape[1]/2-0.5, data.shape[0]/2-0.5)
            aperture = EllipticalAperture(center, a=re, b=re * q, theta=np.deg2rad(pa*u.deg - 90*u.deg))
            aperture.plot(ax=ax, color='gray', lw=2, alpha=0.8, label='Effective Radius')
            ax.scatter(center[1], center[0], color='white', s=100, marker='x', label='Center')

            # Set the axis limits to center a 30x30 region around the image center
            y, x = center
            ax.set_xlim(x - 20, x + 20)
            ax.set_ylim(y - 20, y + 20)
            
#--------------plot radial profiles-----------------
        ax = plt.subplot(2, 3, 4)
        ax.tick_params(axis='both', labelbottom=True, direction='in', which='both', 
                top=True, right=True, left=True, bottom=True, labelsize=13)
        #plot a grey area representing the effective radius
        ax.axvspan(0, re*obj['pixel_length'], color='grey', alpha=0.3, label='Effective Radius')
        ax.errorbar(r, ha, yerr=ha_err, label='Ha', fmt='o:', color=color2, markersize=2,
            capsize=4, capthick=1.5, elinewidth=1.5)
        #plot halimit as horizontal line
        ax.plot(r, ha_lim, color='darkred', linestyle='-.', label='Ha Limit (alt)')

        ax.set_xlim(-0.2, 10)
        ax.set_xlabel('Distance (kpc)')
        ax.set_title('$H\\alpha$ Surface Brightness')
        ax.set_yscale('log')
        ax.legend()

        #plot the radial profile of ha and hb s/n
        ax = plt.subplot(2, 3, 5)
        ax.tick_params(axis='both', labelbottom=True, direction='in', which='both', 
                top=True, right=True, left=True, bottom=True, labelsize=13)
        #plot a grey area representing the effective radius
        ax.axvspan(0, re*obj['pixel_length'], color='grey', alpha=0.3, label='Effective Radius')
        ax.errorbar(r, hb, yerr=hb_err, label='Hb', fmt='o:', color=color1, markersize=2,
            capsize=4, capthick=1.5, elinewidth=1.5)
        #plot hblimit as horizontal line
        ax.plot(r, hb_lim, color='darkgreen', linestyle='-.', label='Hb Limit (alt)')
                                        
        ax.set_xlim(-0.2, 10)
        ax.set_xlabel('Distance (kpc)')
        ax.set_title('$H\\beta$ Surface Brightness')
        ax.set_yscale('log')
        ax.legend()

        #plot the radial profile of balmer decrement
        ax = plt.subplot(2, 3, 6)
        ax.tick_params(axis='both', labelbottom=True, direction='in', which='both', 
                top=True, right=True, left=True, bottom=True, labelsize=13)
        ax.axvspan(0, re*obj['pixel_length'], color='grey', alpha=0.3, label='Effective Radius')
        ax.errorbar(
            r, balmer, yerr=balmer_err, label='Balmer Decrement',
            fmt='o:', color=color3, markersize=4,
            capsize=4, capthick=1.5, elinewidth=1.5
        )
        ax.axhline(y=2.86, color='green', linestyle='--', label='theory')
        ax.fill_between(r, 2.86*0.8, 2.86*1.2, color='green', alpha=0.2)
        ax.set_xlim(-0.2, 8)
        ax.set_ylim(0, np.nanmin([np.nanmax(balmer)+3,25]))
        ax.set_xlabel('Distance [kpc]')
        ax.set_title('Balmer Decrement Ha/Hb')
        plt.tight_layout()
        plt.show()

profile_fits_path = '/Users/yangcheng/clear_bd/Balmer-Decrement-CLEAR/output/data_radial_profiles/gn2_10653_profile.fits'

extracted_fits_path = '/Users/yangcheng/clear_bd/Balmer-Decrement-CLEAR/output/data_extracted/gn2_10653_extracted.fits'

plot_for_obj(extracted_fits_path, profile_fits_path)